In [182]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt

In [194]:
def J(w,yi,xi):
    j = 1/2*np.matmul(np.transpose(w),w)
    for i in range(n):
        j += C*max(0,1-y[i]*np.matmul(np.transpose(w),x[i]))
    return j

def gradJ(w,yi,xi):
    gj = np.transpose(w)
    for i in range(n):
        if(1-y[i]*np.matmul(np.transpose(w),x[i]) > 0):
            gj = gj -1*C * y[i]*x[i]
    print(gj)
    return np.transpose(gj)

def rateThr(r0,t):
    R = np.array([0.01,0.005,0.0025,0])
    return R[t]

def ratePracA(r0,t):
    a = 0.1
    return(r0/(1+r0*t/a))
    
def ratePracB(r0,t):
    return(r0/(1+t))

In [195]:
def ssgd(w0,X,C,rFunc,r0,T):
    w0 = np.transpose(w0)
    w = w0
    r = r0
    m = X.shape[0]
    k = 1
    for t in range(T):
        #print(t)
        #print(r)
        np.random.shuffle(X)
        for j in range(m):
            #print(k) #Uncomment for theory problem
            #print(w) #Uncomment for theory problem
            xj = X[j,:-1]
            #print(xj)
            yj = X[j,-1]
            #print(yj)
            if(yj*np.dot(w,xj) <= 1):
                #print(yj*np.matmul(w,xj))
                #print(yj*xj)
                w = w - r*(w0-C*m*yj*xj)
                #print(w)
            else:
                w = (1-r)*w0
            k += 1
        r = rFunc(r,t)
    return w

def testSSGD(w,X):
    N = X.shape[0]
    n = 0
    for i in range(N):
        predict = 0
        if(np.dot(w,X[i,:-1]) >= 0):
            predict = 1
        else:
            predict = -1
        if(predict == X[i,-1]):
            n += 1
    return n/N

In [103]:
#Used for testing theory problem
#Be sure to uncomment print lines to get correct output.
w0 = np.array(np.mat('0; 0; 0; 0'))
X = np.array(np.mat('1 0.5 -1 0.3 1; 1 -1 -2 -2 -1; 1 1.5 0.2 -2.5 1'))
n = 3
r0 = 0.01
w = ssgd(w0,X,1,ratePrac,r0,3)

0.01
1
[0.]
[ 1.   0.5 -1.   0.3]
[[ 0.03   0.015 -0.03   0.009]]
2
[-0.057]
[-1.  1.  2.  2.]
[[0.    0.045 0.03  0.069]]
3
[-0.099]
[ 1.   1.5  0.2 -2.5]
[[ 0.03   0.09   0.036 -0.006]]
[0.005]
4
[0.0372]
[ 1.   0.5 -1.   0.3]
[[ 0.045   0.0975  0.021  -0.0015]]
5
[0.0915]
[-1.  1.  2.  2.]
[[0.03   0.1125 0.051  0.0285]]
6
[0.1377]
[ 1.   1.5  0.2 -2.5]
[[ 0.045  0.135  0.054 -0.009]]
[0.0025]
7
[0.0558]
[ 1.   0.5 -1.   0.3]
[[ 0.0525   0.13875  0.0465  -0.00675]]
8
[0.16575]
[-1.  1.  2.  2.]
[[0.045   0.14625 0.0615  0.00825]]
9
[0.25605]
[ 1.   1.5  0.2 -2.5]
[[ 0.0525  0.1575  0.063  -0.0105]]


In [222]:
#Load in and clean up data
global trainData
trainData = np.genfromtxt('bank-note/train.csv',delimiter = ',')
b = np.ones(trainData.shape[0])
trainData = np.insert(trainData,0,b,axis=1)
trainData[:,-1][trainData[:,-1] == 0] = -1
global testData 
testData = np.genfromtxt('bank-note/test.csv',delimiter = ',')
b = np.ones(testData.shape[0])
testData = np.insert(testData,0,b,axis=1)
testData[:,-1][testData[:,-1] == 0] = -1

#print(trainData.shape)
w0 = np.zeros((trainData.shape[1]-1,1))
#print(w)
r0 = 0.01
T = 100

In [200]:
#2a
C = 100/873
w = ssgd(w0,trainData,C,ratePracA,r0,T)
print("C = 100/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 500/873
w = ssgd(w0,trainData,C,ratePracA,r0,T)
print("C = 500/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 700/873
w = ssgd(w0,trainData,C,ratePracA,r0,T)
print("C = 700/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 100/873: [[-0.00205483 -0.02203521 -0.00414563 -0.00433416 -0.01146624]] 0.8715596330275229 0.876
C = 500/873: [[ 0.02054833 -0.14545405  0.03843884 -0.16231228 -0.07132124]] 0.5802752293577982 0.572
C = 700/873: [[-0.0431515  -0.15567048 -0.08589882 -0.04849653 -0.00238052]] 0.8853211009174312 0.878


In [201]:
#2b
C = 100/873
w = ssgd(w0,trainData,C,ratePracB,r0,T)
print("C = 100/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 500/873
w = ssgd(w0,trainData,C,ratePracB,r0,T)
print("C = 500/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 700/873
w = ssgd(w0,trainData,C,ratePracB,r0,T)
print("C = 700/873:",w,testSSGD(w,trainData),testSSGD(w,testData))

C = 100/873: [[-0.00236813 -0.03892002 -0.05311799  0.00691495  0.00204509]] 0.6938073394495413 0.69
C = 500/873: [[-0.00267456 -0.03505626 -0.04801007  0.00301076  0.00298516]] 0.6995412844036697 0.702
C = 700/873: [[-0.00064224 -0.01008153 -0.01497289  0.00162733  0.00105151]] 0.6869266055045872 0.68


In [250]:
def dualSVM(a,args):
    #print(a)
    kernel = args
    X = trainData
    n = len(a)
    result = 0
    for i in range(n):
        xi = X[i,:-1]
        yi = X[i,-1]
        for j in range(n):
            xj = X[j,:-1]
            yj = X[j,-1]
            result += a[i]*a[j]*yi*yj*kernel(xi,xj)
        result = result/2
        result += a[i]
    return -1*result

def con1(a):
    y = trainData[:,-1]
    n = len(y)
    result = 0
    for i in range(n):
        result += a[i]*y[i]
    return result

def con2(a):
    n = len(a)
    for i in range(n):
        if(a[i] < 0 or a[i] > C):
            return 1
    return 0

def dotKernel(xi,xj):
    return np.dot(xi,xj)

In [ ]:
a0 = np.zeros((1,trainData.shape[0]))
C = 100/873
cons = ({'type': 'ineq', 'fun': con1},
        {'type': 'ineq', 'fun': con2})
a = opt.minimize(dualSVM,a0,args=(dotKernel),
                 constraints = cons,method = 'SLSQP',tol = 1e-6,
                 options={'disp':True,'maxiter':1})
print('a:',a)
w = np.zeros((1,5))
for i in range(trainData.shape[0]):
    w += a[i]*trainData[i,-1]*trainData[i,:-1]
print(w)